In [2]:
import ccxt
import pandas as pd

In [36]:
exchange = ccxt.ftx()
symbol = 'BTC/USDT'
df_bid = pd.DataFrame(exchange.fetch_order_book(symbol, 10000)['bids'], columns=['price', 'quantity'])
df_ask = pd.DataFrame(exchange.fetch_order_book(symbol, 50)['asks'], columns=['price', 'quantity'])
df_bid['usd_quantity'] = df_bid['price'] *  df_bid['quantity']
df_ask['usd_quantity'] = df_ask['price'] *  df_ask['quantity']

In [37]:
df_bid

,price,quantity,usd_quantity
0,44532.0,0.4050,18035.4600
1,44531.0,0.3122,13902.5782
2,44529.0,0.2244,9992.3076
3,44525.0,0.3244,14443.9100
4,44523.0,0.0065,289.3995
...,...,...,...
95,44338.0,0.0112,496.5856
96,44336.0,0.0005,22.1680
97,44333.0,0.0069,305.8977
98,44332.0,0.0002,8.8664


In [20]:
def get_spread_from_order_book(exchange, symbol, usd_amount, show_order=False):
    df_bid = pd.DataFrame(exchange.fetch_order_book(symbol, 5000)['bids'], columns=['price', 'quantity'])
    df_ask = pd.DataFrame(exchange.fetch_order_book(symbol, 5000)['asks'], columns=['price', 'quantity'])
    df_bid['usd_quantity'] = df_bid['price'] *  df_bid['quantity']
    df_ask['usd_quantity'] = df_ask['price'] *  df_ask['quantity']

    i = 0
 
    for j in range(0,2,1): 
        operations_array = []
        usd_spent = 0
        if j == 0:
            df = df_ask.copy()
            # print(df)
        else:
            df = df_bid.copy()  
            # print(df)
        while(usd_spent < usd_amount):
            if df.iloc[i]['usd_quantity'] > usd_amount - usd_spent:
                operations_array.append({"amount":usd_amount - usd_spent, "price":df.iloc[i]['price']})
                usd_spent += (usd_amount - usd_spent)
            else: 
                operations_array.append({"amount":df.iloc[i]['usd_quantity'], "price":df.iloc[i]['price']})
                usd_spent += df.iloc[i]['usd_quantity']
            i += 1
            
        dt = pd.DataFrame(operations_array)
        dt['sum'] = dt['amount'] * dt['price']
        mean_price = dt['sum'].sum() / dt['amount'].sum()
        mid_bid_ask_price = (df_bid.iloc[0]["price"]+df_ask.iloc[0]["price"])/2
        diff_spread = abs(mid_bid_ask_price - mean_price) / mean_price
        spread_fee = usd_amount * diff_spread
        del dt['sum']
        if j == 0:
            print("||| Mid Bid Ask Price :",mid_bid_ask_price,"$ |||")
            print("\n------------ BUY ------------")
        else:
            print("\n------------ SELL ------------")
        
        print("||| Mean Operation Price  =",mean_price,"$ |||")
        # print("||| Mid Bid Ask Price =", mid_bid_ask_price,"$ |||")
        print("||| Spread Differance =", round(diff_spread*100,4),"% |||")
        print("||| Spread Fees =", round(spread_fee,2),"$ |||")
        if show_order:
            print("-"*30)
            print("--- Operation array ---")
            print(dt.to_string(index=False))


exchange = ccxt.binance()
get_spread_from_order_book(
    exchange = exchange,
    symbol = "GRT/USDT", 
    usd_amount = 100000,
    show_order = False
)
        

||| Mid Bid Ask Price : 0.33045 $ |||

------------ BUY ------------
||| Mean Operation Price  = 0.3310672613249999 $ |||
||| Spread Differance = 0.1864 % |||
||| Spread Fees = 186.45 $ |||

------------ SELL ------------
||| Mean Operation Price  = 0.328353781809 $ |||
||| Spread Differance = 0.6384 % |||
||| Spread Fees = 638.4 $ |||
